In [ ]:
import pandas as pd
# from sklearn import datasets
from datetime import date

# Reading Bronze Data
df = spark.read.table("bronze_data.fact_trade")
pdf = df.toPandas()

# Sorting the big data source by HS_CODE
pdf = pdf.sort_values(by='HS_CODE', ascending=True)
# display(pdf)

# Reading HS Description table
Product_info = spark.read.table("bronze_data.dim_hs_product")
HS_DESC = Product_info.toPandas() 


# Make sure HS_CODE is numeric (just in case it's stored as string)
HS_DESC['HS_CODE'] = pd.to_numeric(HS_DESC['HS_CODE'], errors='coerce')

# Sort by HS_CODE (ascending order)
# HS_DESC = HS_DESC.sort_values(by='HS_CODE', ascending=True)


StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 3, Finished, Available, Finished)

#### **Getting HS_Product Details**

In [ ]:
# display(HS_DESC)
pdf = pdf.merge(HS_DESC[['HS_CODE', 'DESCRIPTION']], on='HS_CODE', how='left')
# display(pdf)

StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 4, Finished, Available, Finished)

#### **Creating Data frames**

In [ ]:
import_Trades = []
export_Trades = []
# display(HS_DESC)
# Traverse through HS_CODE groups
# Optionally set HS_CODE as index for faster lookup
# HS_DESC_indexed = HS_DESC.set_index('HS_CODE')


for index, row in pdf.iterrows():
    # print("HS_CODE:", row['HS_CODE'])
    # print("FLOW_CODE:", row['FLOW_CODE'])
    if row['FLOW_CODE'] == 4:
       
        import_Trades.append({
            'HS_CODE': row['HS_CODE'],
             'HS_DESC': row['DESCRIPTION'],
            'TRADE_DATE': row['TRADE_DATE'],
            'VALUE': row['VALUE'],   # optional if VALUE column exists
            'FLOW_CODE': 'Import'
             ,'PARTNER_KEY': row['PARTNER_KEY']
    
        })
    else:
        export_Trades.append({
            'HS_CODE': row['HS_CODE'],
             'HS_DESC': row['DESCRIPTION'],
             'TRADE_DATE': row['TRADE_DATE'],
            'VALUE': row['VALUE'],   # optional if VALUE column exists
            'FLOW_CODE': 'Export'
            ,'PARTNER_KEY': row['PARTNER_KEY']
        })

imported_df = pd.DataFrame(import_Trades)
exported_df = pd.DataFrame(export_Trades)
# display(exported_df)


StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 5, Finished, Available, Finished)

In [ ]:
sample_df  = imported_df.groupby(['HS_CODE','HS_DESC', 'PARTNER_KEY', 'TRADE_DATE'], as_index=False)['VALUE'].sum()
display(sample_df)

StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6b53c2e3-48d6-403c-8b35-d4ee40940c52)

In [ ]:
import pandas as pd
import numpy as np

sample_df['TRADE_DATE'] = pd.to_datetime(sample_df['TRADE_DATE'], errors='coerce')
sample_df['VALUE'] = pd.to_numeric(sample_df['VALUE'], errors='coerce').fillna(0)

# Extract calendar units
sample_df['year_month'] = sample_df['TRADE_DATE'].dt.to_period('M')
sample_df['quarter'] = sample_df['TRADE_DATE'].dt.to_period('Q')

# Aggregate monthly totals
grouped_df = (
    sample_df.groupby(['HS_CODE','year_month','PARTNER_KEY'], as_index=False)['VALUE']
             .sum()
             .sort_values(['HS_CODE','PARTNER_KEY','year_month'])
)

# Convert year_month → timestamp
grouped_df['year_month'] = grouped_df['year_month'].dt.to_timestamp()

# Create "previous" month/year/quarter columns for merging
grouped_df['prev_month']  = grouped_df['year_month'] - pd.DateOffset(months=1)
grouped_df['prev_year']   = grouped_df['year_month'] - pd.DateOffset(years=1)
grouped_df['prev_quarter'] = grouped_df['year_month'] - pd.DateOffset(months=4)
# display(grouped_df)


StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 7, Finished, Available, Finished)

In [ ]:
# Merge previous month
df_prev_month = grouped_df[['HS_CODE','PARTNER_KEY','year_month','VALUE']].rename(
    columns={'year_month':'prev_month','VALUE':'prev_month_value'}
)

grouped_df = grouped_df.merge(df_prev_month,
                              on=['HS_CODE','PARTNER_KEY','prev_month'],
                              how='left')

# Merge previous year
df_prev_year = grouped_df[['HS_CODE','PARTNER_KEY','year_month','VALUE']].rename(
    columns={'year_month':'prev_year','VALUE':'prev_year_value'}
)

grouped_df = grouped_df.merge(df_prev_year,
                              on=['HS_CODE','PARTNER_KEY','prev_year'],
                              how='left')

# Merge previous quarter
df_prev_quarter = grouped_df[['HS_CODE','PARTNER_KEY','year_month','VALUE']].rename(
    columns={'year_month':'prev_quarter','VALUE':'prev_quarter_value'}
)

grouped_df = grouped_df.merge(df_prev_quarter,
                              on=['HS_CODE','PARTNER_KEY','prev_quarter'],
                              how='left')

# display(grouped_df)


StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 8, Finished, Available, Finished)

In [ ]:
# Replace missing with NaN (first month will be NaN)
grouped_df['prev_month_value'] = grouped_df['prev_month_value'].astype(float)
grouped_df['prev_year_value'] = grouped_df['prev_year_value'].astype(float)
grouped_df['prev_quarter_value'] = grouped_df['prev_quarter_value'].astype(float)

# Calculate MoM, YoY, QoQ
grouped_df['MOM_PERCENT_IM'] = (((grouped_df['VALUE'] - grouped_df['prev_month_value'])
                                / grouped_df['prev_month_value']) * 100).round(4)

grouped_df['YOY_PERCENT_IM'] = (((grouped_df['VALUE'] - grouped_df['prev_year_value'])
                                / grouped_df['prev_year_value']) * 100).round(4)

grouped_df['QOQ_PERCENT_IM'] = (((grouped_df['VALUE'] - grouped_df['prev_quarter_value'])
                                / grouped_df['prev_quarter_value']) * 100).round(4)

# Monthly difference
grouped_df['MONTHLY_DIFF'] = grouped_df['VALUE'] - grouped_df['prev_month_value']

# Drop helper columns
grouped_df = grouped_df.drop(columns=['prev_month_value', 'prev_year_value', 'prev_month', 'prev_quarter', 'prev_year','prev_quarter_value'])
# Replace infinite or missing values
grouped_df = grouped_df.replace([np.inf, -np.inf], np.nan).fillna(0)

# print(len(df))
display(grouped_df)


StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a2b2a6c7-b69f-45bb-8282-3900c615e344)

In [ ]:
# # Drop existing table if it exists
# spark.sql("DROP TABLE IF EXISTS Statistical_data.stat_total_import")
# # Drop existing table if it exists 
# spark.sql("DROP TABLE IF EXISTS Statistical_data.stat_total_export")

# Drop existing table if it exists
spark.sql("DROP TABLE IF EXISTS Statistical_data.stat_hs_partner_import")
# Drop existing table if it exists 
spark.sql("DROP TABLE IF EXISTS Statistical_data.stat_hs_partner_export")

# Drop existing table if it exists
spark.sql("DROP TABLE IF EXISTS Statistical_data.stat_hs_grouped_import")
# Drop existing table if it exists 
spark.sql("DROP TABLE IF EXISTS Statistical_data.stat_hs_grouped_export")


StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 10, Finished, Available, Finished)

DataFrame[]

In [ ]:
# # make a copy so original grouped_df stays untouched
# country_level_df = grouped_df.copy()

# # replace the partner key with 'USA'
# # country_level_df['PARTNER_KEY'] = 'USA'



# country_level_df = (
#     country_level_df
#         .groupby(['HS_CODE','year_month','MOM_PERCENT_IM'], as_index=False)['VALUE']
#         .sum()
# )
# display(country_level_df)

StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 11, Finished, Available, Finished)

In [ ]:
hs1_df = grouped_df[grouped_df['HS_CODE'] == 1].copy()
display(hs1_df)


StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 35257d0d-da75-425b-a597-d79e66631f37)

In [ ]:
grouped_df_spark = spark.createDataFrame(grouped_df)
hsc_grouped_spark = spark.createDataFrame(hs1_df)

hsc_grouped_spark.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("Statistical_data.stat_hs_grouped_import")

grouped_df_spark.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("Statistical_data.stat_hs_partner_import")

# spark.sql("OPTIMIZE import_trade_stats ZORDER BY (HS_CODE, TRADE_DATE)")


StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 13, Finished, Available, Finished)

In [ ]:
Sample_ex_df  = exported_df.groupby(['HS_CODE','HS_DESC', 'PARTNER_KEY', 'TRADE_DATE'], as_index=False)['VALUE'].sum()
# display(Sample_ex_df)

StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 14, Finished, Available, Finished)

#### **Aggregated Exported Table without partner key**

In [ ]:
import pandas as pd
import numpy as np

Sample_ex_df['TRADE_DATE'] = pd.to_datetime(sample_df['TRADE_DATE'], errors='coerce')
Sample_ex_df['VALUE'] = pd.to_numeric(sample_df['VALUE'], errors='coerce').fillna(0)

# Extract calendar units
Sample_ex_df['year_month'] = Sample_ex_df['TRADE_DATE'].dt.to_period('M')
Sample_ex_df['quarter'] = Sample_ex_df['TRADE_DATE'].dt.to_period('Q')

# Aggregate monthly totals
grouped_ex_df = (
    Sample_ex_df.groupby(['HS_CODE','year_month','PARTNER_KEY'], as_index=False)['VALUE']
             .sum()
             .sort_values(['HS_CODE','PARTNER_KEY','year_month'])
)

# Convert year_month → timestamp
grouped_ex_df['year_month'] = grouped_ex_df['year_month'].dt.to_timestamp()

# Create "previous" month/year/quarter columns for merging
grouped_ex_df['prev_month']  = grouped_ex_df['year_month'] - pd.DateOffset(months=1)
grouped_ex_df['prev_year']   = grouped_ex_df['year_month'] - pd.DateOffset(years=1)
grouped_ex_df['prev_quarter'] = grouped_ex_df['year_month'] - pd.DateOffset(months=4)
# display(grouped_df)

# Merge previous month
df_prev_month = grouped_ex_df[['HS_CODE','PARTNER_KEY','year_month','VALUE']].rename(
    columns={'year_month':'prev_month','VALUE':'prev_month_value'}
)

grouped_ex_df = grouped_ex_df.merge(df_prev_month,
                              on=['HS_CODE','PARTNER_KEY','prev_month'],
                              how='left')

# Merge previous year
df_prev_year = grouped_ex_df[['HS_CODE','PARTNER_KEY','year_month','VALUE']].rename(
    columns={'year_month':'prev_year','VALUE':'prev_year_value'}
)

grouped_ex_df = grouped_ex_df.merge(df_prev_year,
                              on=['HS_CODE','PARTNER_KEY','prev_year'],
                              how='left')

# Merge previous quarter
df_prev_quarter = grouped_ex_df[['HS_CODE','PARTNER_KEY','year_month','VALUE']].rename(
    columns={'year_month':'prev_quarter','VALUE':'prev_quarter_value'}
)

grouped_ex_df = grouped_ex_df.merge(df_prev_quarter,
                              on=['HS_CODE','PARTNER_KEY','prev_quarter'],
                              how='left')

# display(grouped_df)

# Replace missing with NaN (first month will be NaN)
grouped_ex_df['prev_month_value'] = grouped_ex_df['prev_month_value'].astype(float)
grouped_ex_df['prev_year_value'] = grouped_ex_df['prev_year_value'].astype(float)
grouped_ex_df['prev_quarter_value'] = grouped_ex_df['prev_quarter_value'].astype(float)

# Calculate MoM, YoY, QoQ
grouped_ex_df['MOM_PERCENT_EX'] = (((grouped_ex_df['VALUE'] - grouped_ex_df['prev_month_value'])
                                / grouped_ex_df['prev_month_value']) * 100).round(4)

grouped_ex_df['YOY_PERCENT_EX'] = (((grouped_ex_df['VALUE'] - grouped_ex_df['prev_year_value'])
                                / grouped_ex_df['prev_year_value']) * 100).round(4)

grouped_ex_df['QOQ_PERCENT_EX'] = (((grouped_ex_df['VALUE'] - grouped_ex_df['prev_quarter_value'])
                                / grouped_ex_df['prev_quarter_value']) * 100).round(4)

# Monthly difference
grouped_ex_df['MONTHLY_DIFF'] = grouped_ex_df['VALUE'] - grouped_ex_df['prev_month_value']

# Drop helper columns
grouped_ex_df = grouped_ex_df.drop(columns=['prev_month_value', 'prev_year_value', 'prev_month', 'prev_quarter', 'prev_year','prev_quarter_value'])
# Replace infinite or missing values
grouped_ex_df = grouped_ex_df.replace([np.inf, -np.inf], np.nan).fillna(0)

display(grouped_ex_df)




StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 20a62530-3fba-4bf8-b34f-d80045ab8565)

In [ ]:
hs1_ex_df = grouped_ex_df[grouped_ex_df['HS_CODE'] == 1].copy()
display(hs1_ex_df)

StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fe4b78d1-ce76-4d58-a6c2-2f19c4eac53d)

In [ ]:
grouped_ex_df_spark = spark.createDataFrame(grouped_ex_df)

grouped_ex_df_spark.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("Statistical_data.stat_hs_partner_export")


hs1_ex_df_spark = spark.createDataFrame(hs1_ex_df)

hs1_ex_df_spark.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("Statistical_data.stat_hs_grouped_export")


# spark.sql("OPTIMIZE import_trade_stats ZORDER BY (HS_CODE, TRADE_DATE)")


StatementMeta(, 28492ed5-c8a1-4541-8b0c-05218c91792b, 17, Finished, Available, Finished)